In [1]:
import grequests
import geopandas as gpd
from datetime import datetime
import pandas as pd

C:\Users\halpe\anaconda3\envs\trees\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
state2Digit = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [3]:
urls = [f"https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=integrated_county_timeseries_state_{stateCode}_external" for stateCode in state2Digit]
breakpoint = 12
urlDict = []
for i in range(0,5):
    urlDict.append(urls[breakpoint*i:breakpoint*(i+1)])

In [5]:
responses = []
for urlList in urlDict:
    rs = (grequests.get(u,  timeout=120) for u in urlList)
    response = grequests.map(rs)
    responses.append(response)

In [6]:
parsed = ''

for responseSet in responses:
    for response in responseSet:
        if len(parsed)==0:
            parsed = pd.DataFrame(response.json()['integrated_county_timeseries_external_data'])
        else:
            parsed = pd.concat([parsed, pd.DataFrame(response.json()['integrated_county_timeseries_external_data'])])

parsed = parsed.sort_values('date')
parsed['date'] = parsed['date'].str.slice(0,-9)

In [86]:
countyPops = pd.read_csv('county_populations.csv')[['GEOID','state','population']]

In [28]:
countyPops = parsed[['fips_code', 'state']].merge(pd.read_csv('county_populations.csv')[["GEOID",'population']], 
           how="left", left_on="fips_code", right_on="GEOID")[['fips_code','state','population']].drop_duplicates()

In [59]:
stateFips = countyPops[['fips_code','state']]
stateFips.loc[stateFips.fips_code.notnull(),'fips_code'] = stateFips['fips_code'].astype(str).str.slice(0,-3)
stateFips = stateFips.drop_duplicates()

In [7]:
colsToParse = [
    {
        'column':'new_cases_7_day_rolling_average',
        'csv':'covid_confirmed_cdc'
    },
    {
        'column':'new_deaths_7_day_rolling_average', 
        'csv':'covid_deaths_cdc', 
    },
    {
        'column':'new_test_results_reported_7_day_rolling_average', 
        'csv':'covid_testing_cdc'
    },
    {
        'column':'percent_new_test_results_reported_positive_7_day_rolling_average',
        'csv':'covid_wk_pos_cdc'
    }]

In [32]:
dfs = []
for entry in colsToParse:
    colName = entry['column']
    # thanks to @piRSquared on stackoverflow for this nifty pivot expressions
    # https://stackoverflow.com/questions/54915215/expressing-time-series-data-in-the-columns-rather-than-the-rows-of-a-dataframe
    tempDf = parsed[['fips_code','date',colName]]
    tempDf = tempDf.pivot_table(index='fips_code', columns='date').swaplevel(0, 1, 1).sort_index(1).reset_index()
    tempDf.columns = [column[0] for column in list(tempDf.columns)]
    dfs.append(tempDf)
#     tempDf.to_csv(f'./csv/{entry["csv"]}.csv')

new_cases_7_day_rolling_average
new_deaths_7_day_rolling_average
new_test_results_reported_7_day_rolling_average
percent_new_test_results_reported_positive_7_day_rolling_average


In [79]:
def getWeightedResults(df):
    joined = df.merge(countyPops, on="fips_code", how="left")
    joined = joined.merge(joined[['state','population']].groupby('state') \
    .sum().rename(columns={'population': 'statePopulation'}).reset_index(),
                      how="left", on="state")
    joined['proportion'] = joined['population']/joined['statePopulation']
    
    dateColumns = test_results.columns[1:-4]

    for column in dateColumns:
        joined[column] = joined[column] * joined['proportion']

    weighted_results = joined[['state'] + list(dateColumns)].groupby('state').sum() \
        .reset_index().merge(stateFips, how="left", on="state")[['fips_code'] + list(dateColumns)]
    
    return weighted_results

In [34]:
test_results = dfs[2].merge(countyPops, on="fips_code", how="left")

In [42]:
test_results = test_results.merge(test_results[['state','population']].groupby('state') \
.sum().rename(columns={'population': 'statePopulation'}).reset_index(),
                  how="left", on="state")

In [45]:
test_results.head()

,fips_code,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2021-01-27,2021-01-28,2021-01-29,2021-01-30,2021-01-31,2021-02-01,state,population,statePopulation,proportion
0,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,161.43,146.29,142.86,134.00,122.14,0.0,AL,55200,4864680,0.011347
1,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,530.71,491.43,452.29,447.29,441.14,0.0,AL,208107,4864680,0.042779
2,1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,42.14,38.57,33.29,25.57,24.86,0.0,AL,25782,4864680,0.005300
3,1007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,58.00,52.86,46.86,44.00,38.57,0.0,AL,22527,4864680,0.004631
4,1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,102.00,99.00,90.14,93.71,93.43,0.0,AL,57645,4864680,0.011850


In [44]:
test_results['proportion'] = test_results['population']/test_results['statePopulation']

In [70]:
dateColumns = test_results.columns[1:-4]

for column in dateColumns:
    test_results[column] = test_results[column] * test_results['proportion']

weighted_results = test_results[['state'] + list(dateColumns)].groupby('state').sum() \
    .reset_index().merge(stateFips, how="left", on="state")[['fips_code'] + list(dateColumns)]

In [71]:
weighted_results

,fips_code,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2021-01-23,2021-01-24,2021-01-25,2021-01-26,2021-01-27,2021-01-28,2021-01-29,2021-01-30,2021-01-31,2021-02-01
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29.413999,28.781894,29.083250,30.279873,31.267657,31.285008,32.391948,30.967556,30.079973,0.000000
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.126233,0.133103,0.130458,0.130961,0.128687,0.122397,0.106799,0.105620,0.100225,0.000000
2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.070567,0.070360,0.073710,0.073869,0.072426,0.073584,0.072509,0.079336,0.079993,0.072667
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2019.601956,2031.902991,2035.571669,2009.151870,1958.974789,1947.726852,1905.336298,1855.916994,1783.189912,1706.184443
4,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,77.684081,77.525130,72.837670,62.817680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.365797,0.357174,0.341610,0.351695,0.331677,0.302461,0.289615,0.284310,0.280682,0.273846
6,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28.778039,29.873204,31.965138,31.967591,28.150642,22.872113,0.000000,0.000000,0.000000,0.000000
7,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6953.710000,6699.140000,6335.290000,6304.000000,6858.290000,7434.140000,7404.000000,7247.000000,7121.290000,6836.430000
8,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,391.326433,397.180606,375.760989,370.702544,362.464883,362.564539,335.578439,325.791630,308.613315,282.892319
9,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.192285,1.195424,1.190340,1.202855,1.214897,1.178212,1.142092,1.100054,1.045633,0.962403
